In [ ]:
import os
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
def setup_driver(download_dir):
    options = Options()
    options.add_experimental_option("prefs", {
        "download.default_directory": os.path.abspath(download_dir),
        "download.prompt_for_download": False,
        "directory_upgrade": True,
        "safebrowsing.enabled": True
    })
    options.add_argument("--start-maximized")
    return webdriver.Chrome(options=options)
def click_folder(driver, label_text):
    try:
        print(f":mag: Searching for: {label_text}")
        xpath = f"//*[contains(text(), '{label_text}')]"
        elements = driver.find_elements(By.XPATH, xpath)
        print(f":receipt: Found {len(elements)} element(s) with '{label_text}'")
        for el in elements:
            try:
                print(f":mag_right: Trying: <{el.tag_name}> → '{el.text.strip()}'")
                driver.execute_script("arguments[0].scrollIntoView(true);", el)
                time.sleep(1)
                el.click()
                print(f":white_check_mark: Clicked on: {label_text}")
                return True
            except Exception as e:
                print(f":warning: Failed to click: {e}")
        raise Exception("No clickable element worked.")
    except Exception as e:
        print(f":x: Failed to click '{label_text}': {e}")
        return False
def wait_for_download(download_dir, filename, timeout=120):
    print(f":hourglass_flowing_sand: Waiting for download: {filename}")
    for _ in range(timeout):
        files = os.listdir(download_dir)
        if filename in files and not filename.endswith(".crdownload"):
            print(f":white_check_mark: Download complete: {filename}")
            return True
        time.sleep(2)
    print(":x: Download timed out.")
    return False
def main():
    year = "2024"
    month = "01"
    filename = f"DA-LMP-MONTHLY-SL-{year}{month}.csv"
    download_dir = os.path.join(os.getcwd(), "spp_downloads")
    os.makedirs(download_dir, exist_ok=True)
    driver = setup_driver(download_dir)
    try:
        print(":globe_with_meridians: Opening SPP LMP Portal...")
        driver.get("https://portal.spp.org/pages/da-lmp-by-location")
        time.sleep(6)
        # Step 1: Click year folder
        if not click_folder(driver, year):
            raise Exception(":x: Could not open year folder.")
        time.sleep(3)
        # Step 2: Click month folder
        if not click_folder(driver, month):
            raise Exception(":x: Could not open month folder.")
        time.sleep(3)
        # Step 3: Click the CSV file
        if not click_folder(driver, filename):
            raise Exception(":x: Could not find or click the CSV file.")
        # Step 4: Wait for download to complete
        if not wait_for_download(download_dir, filename):
            raise Exception(":x: File was not downloaded within the timeout.")
    except Exception as e:
        print(f":octagonal_sign: Error: {e}")
    finally:
        input(":mag: Press Enter to quit browser...")
        driver.quit()
if __name__ == "__main__":
    main()
    

:globe_with_meridians: Opening SPP LMP Portal...
:mag: Searching for: 2024
:receipt: Found 1 element(s) with '2024'
:mag_right: Trying: <span> → '2024'
:white_check_mark: Clicked on: 2024
:mag: Searching for: 01
:receipt: Found 2 element(s) with '01'
:mag_right: Trying: <span> → '01'
:white_check_mark: Clicked on: 01
:mag: Searching for: DA-LMP-MONTHLY-SL-202401.csv
:receipt: Found 1 element(s) with 'DA-LMP-MONTHLY-SL-202401.csv'
:mag_right: Trying: <div> → 'DA-LMP-MONTHLY-SL-202401.csv'
:white_check_mark: Clicked on: DA-LMP-MONTHLY-SL-202401.csv
:hourglass_flowing_sand: Waiting for download: DA-LMP-MONTHLY-SL-202401.csv
:white_check_mark: Download complete: DA-LMP-MONTHLY-SL-202401.csv


:mag: Press Enter to quit browser... 
